### LIGHTGBM을 활용한 이상거래 탐지

In [3]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
import os
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import gc
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge


In [4]:
os.chdir(r"C:\Users\USER\20190722_아시아 경제 update\notebook\CopyProject")

df = pd.read_csv('example_zigbang.csv')

df.head(5)

,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
0,0,5584,1,200601,11~20,1999,47.43,6,37.585965,127.000231,...,4.0,individual,gas,91120,65.63,46,1.0,1.0,corridor,215000000
1,1,5584,1,200601,11~20,1999,44.37,8,37.585965,127.000231,...,4.0,individual,gas,91119,61.39,10,2.0,1.0,corridor,200000000
2,2,5059,1,200601,11~20,1992,54.70,8,37.580511,127.014016,...,9.0,individual,gas,8430,72.36,201,2.0,1.0,corridor,168000000
3,3,2816,1,200601,11~20,1993,64.66,11,37.580324,127.011788,...,11.0,individual,gas,5839,87.30,284,2.0,1.0,corridor,165000000
4,4,2816,1,200601,11~20,1993,106.62,7,37.580324,127.011788,...,11.0,individual,gas,5836,127.74,112,4.0,2.0,stairway,280000000


In [5]:
df.columns

Index(['key', 'apartment_id', 'city', 'transaction_year_month',
       'transaction_date', 'year_of_completion', 'exclusive_use_area', 'floor',
       'latitude', 'longitude', 'address_by_law',
       'total_parking_capacity_in_site', 'total_household_count_in_sites',
       'apartment_building_count_in_sites', 'tallest_building_in_sites',
       'lowest_building_in_sites', 'heat_type', 'heat_fuel', 'room_id',
       'supply_area', 'total_household_count_of_area_type', 'room_count',
       'bathroom_count', 'front_door_structure', 'transaction_real_price'],
      dtype='object')

### 데이터 전처리 준비작업

*Vaildation 구축: Test가 가장 마지막 거래로만 이루어져 있어서, 
    제 제출 전 점수를 평가 할 validation도 비슷하게 구축. 
    (주의 : 샘플링에 의해 Validation Score는 달라지기 때문에 
     동일한 샘플링 기법을 적용한것 끼리 비교해야 함 !!!)

In [6]:
N = len(df)
ratio = 0.7
np.random.seed(0)
idx_train = np.random.choice(np.arange(N), np.int(ratio * N), replace=False)
idx_test = list(set(np.arange(N)).difference(idx_train))

#교차검증을 위해 train데이터를 70% test=30%로 나눔
# loc은 컬럼명처럼 명을 기준으로 행과 열을 추출 iloc은 index를 기준으로 행과 열 추출

df_train = df.iloc[idx_train]
df_test = df.iloc[idx_test]

In [7]:
df_train.head(5)

,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
1554595,1554813,17782,0,201804,21~30,2006,84.8420,20,35.188764,129.117489,...,20.0,individual,gas,51094,108.88,329,3.0,2.0,stairway,425000000
17701,17701,544,1,200603,11~20,1995,114.8800,7,37.626773,127.082043,...,9.0,individual,gas,1417,139.17,204,4.0,2.0,stairway,350000000
1209273,1209283,5047,1,201603,21~31,2003,84.9600,19,37.489768,126.950618,...,10.0,central,gas,8406,109.42,676,3.0,2.0,stairway,451000000
831528,831531,2895,1,201311,11~20,1998,39.8400,10,37.631771,127.050187,...,12.0,district,cogeneration,6129,54.83,89,1.0,1.0,corridor,147000000
424138,424138,8001,0,200908,21~31,2005,84.6528,26,35.178213,129.122049,...,30.0,individual,gas,37516,114.16,270,3.0,2.0,stairway,398000000


In [8]:
train = df_train
test = df_test

In [9]:
test.loc[test['key']==1503614,'city'] = 0

* 데이터가 시간의 순서대로 이루어져 잇어서 Merge과정에서 순서가 깨지지 않도록 index컬름을 
생성해서 sort 작업을 해줌
*(*)이를 안해주면 fold에서 다른 cv값이 나옴

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

train_len = train.shape[0]
df_all = pd.concat([train,test])

index = []
for i in range(0,df_all.shape[0]):
    index.append(i)
df_all['index'] = index

train = df_all[:train_len].reset_index(drop=True)
df_test = df_all[train_len:].reset_index(drop=True)

df_train_busan = train[train['city']==0]
df_test_busan = df_test[df_test['city']==0]
df_train_seoul = train[train['city']==1]
df_test_seoul = df_test[df_test['city']==1]

신뢰성 있는 Validation 구축을 위해서 아파트 별로 가장 마지막 거래를 Validation으로 추출

In [11]:
busan_val_idx = df_train_busan.groupby(['apartment_id']).tail(1).index  #tail 각 그룹의 마지막 n 행을 반환합니다.
busan_valid = df_train_busan.loc[busan_val_idx,:]
busan_valid = busan_valid[['key','transaction_real_price']]

seoul_val_idx = df_train_seoul.groupby(['apartment_id']).tail(1).index
seoul_valid = df_train_seoul.loc[seoul_val_idx,:]
seoul_valid = seoul_valid[['key','transaction_real_price']]

In [12]:
print(busan_valid.shape[0],seoul_valid.shape[0])

1271 2627


In [13]:
del df_train_busan,df_train_seoul,df_test_busan,df_test_seoul
del df_test,train,df_all
gc.collect()

42

### 데이터 전처리
*트레인은 날짜가 고른 반면, 테스트는 2019년도 1월 이후가 압도적으로 많음. 샘플링 작업이 필요.

*방과 화장실 0인 값 대체 : 동일한 아파트에서 비슷한 크기에 값이 존재하면 그로 채워넣고, 그렇지 않으면 비슷한 크기에서 median으로 채워 넣음.

*방과 화장실 결측치 대체 : 동일한 아파트에서 비슷한 크기에 값이 존재하면 그로 채워넣고, 그렇지 않으면 비슷한 크기에서 median으로 채워 넣음.

*주차장의 결측치는 0으로 대체

*난방과 현관구조는 None으로 대체

In [14]:
df = pd.read_csv('example_zigbang.csv')
test_idx = test['key']

N = len(df)
ratio = 0.7
np.random.seed(0)
idx_train = np.random.choice(np.arange(N), np.int(ratio * N), replace=False)
idx_test = list(set(np.arange(N)).difference(idx_train))

df_train = df.iloc[idx_train]
df_test = df.iloc[idx_test]

train = df_train
test = df_test


train_up1 = train[train['transaction_year_month']>201806]
train_up1['transaction_real_price'] = train_up1['transaction_real_price'] + 10000000

train_up2 = train[train['transaction_year_month']>201806]
train_up2['transaction_real_price'] = train_up2['transaction_real_price'] + 5000000

train = pd.concat([train,train_up1])
train = pd.concat([train,train_up2])
train = train.reset_index(drop=True)
del train_up1,train_up2
gc.collect()

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

train_len = train.shape[0]
df_all = pd.concat([train,test])

index = []
for i in range(0,df_all.shape[0]):
    index.append(i)
df_all['index'] = index

In [15]:
df_test.shape

(480438, 25)

In [16]:
### 0인 값 대체
#loc은 컬럼명처럼 명을 기준으로 행과 열을 추출
df_all.loc[(df_all['apartment_id']==2805) & (df_all['supply_area'] > 90),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==2805) & (df_all['supply_area'] > 90),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==895) & (df_all['supply_area'] > 137),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==895) & (df_all['supply_area'] > 137),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==903) & (df_all['supply_area'] > 135),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==903) & (df_all['supply_area'] > 135),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==1622) & (df_all['supply_area'] == 127.07),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==1622) & (df_all['supply_area'] == 127.07),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==14029) & (df_all['supply_area'] > 100),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==14029) & (df_all['supply_area'] > 100),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==14029) & (df_all['supply_area'] < 100),'room_count'] = 2
df_all.loc[(df_all['apartment_id']==14029) & (df_all['supply_area'] < 100),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==12067) & (df_all['supply_area'] >= 95),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==12067) & (df_all['supply_area'] >= 95),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==12067) & (df_all['supply_area'] == 92),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==12067) & (df_all['supply_area'] == 92),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==12067) & (df_all['supply_area'] < 90),'room_count'] = 2
df_all.loc[(df_all['apartment_id']==12067) & (df_all['supply_area'] < 90),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==360) & (df_all['supply_area'] == 189.99),'room_count'] = 5
df_all.loc[(df_all['apartment_id']==360) & (df_all['supply_area'] == 189.99),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==360) & (df_all['supply_area'] == 154.46),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==360) & (df_all['supply_area'] == 154.46),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==568) & (df_all['supply_area']//10 == 11.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==568) & (df_all['supply_area']//10 == 11.0),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==1543) & (df_all['supply_area'] > 150 ),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==1543) & (df_all['supply_area'] > 150),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==618) & (df_all['supply_area'] == 92.94),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==618) & (df_all['supply_area'] == 92.94),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==618) & (df_all['supply_area'] == 110.57),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==618) & (df_all['supply_area'] == 110.57),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==4368) & (df_all['supply_area'] > 90),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==4368) & (df_all['supply_area'] > 90),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==4368) & (df_all['supply_area'] < 90),'room_count'] = 2
df_all.loc[(df_all['apartment_id']==4368) & (df_all['supply_area'] < 90),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==3701) & (df_all['supply_area'] == 148.55),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==3701) & (df_all['supply_area'] == 148.55),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==346) & (df_all['supply_area'] > 100),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==346) & (df_all['supply_area'] > 100),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==1524) & (df_all['supply_area'] == 104.39),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==1524) & (df_all['supply_area'] == 104.39),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==1524) & (df_all['supply_area'] == 175.60),'room_count'] = 5
df_all.loc[(df_all['apartment_id']==1524) & (df_all['supply_area'] == 175.60),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==431) & (df_all['supply_area']//10 == 9.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==431) & (df_all['supply_area']//10 == 9.0),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==431) & (df_all['supply_area']//10 == 10.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==431) & (df_all['supply_area']//10 == 10.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==431) & (df_all['supply_area']//10 == 13.0),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==431) & (df_all['supply_area']//10 == 13.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==431) & (df_all['supply_area']//10 == 14.0),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==431) & (df_all['supply_area']//10 == 14.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==1112) & (df_all['supply_area']//10 == 7.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==1112) & (df_all['supply_area']//10 == 7.0),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==65) & (df_all['supply_area']//10 == 11.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==65) & (df_all['supply_area']//10 == 11.0),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==541) & (df_all['supply_area']//10 == 8.0),'room_count'] = 2
df_all.loc[(df_all['apartment_id']==541) & (df_all['supply_area']//10 == 8.0),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==184) & (df_all['supply_area'] == 66.12),'room_count'] = 2
df_all.loc[(df_all['apartment_id']==184) & (df_all['supply_area'] == 66.12),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==2601) & (df_all['supply_area'] == 104.97),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==2601) & (df_all['supply_area'] == 104.97),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==6161) & (df_all['supply_area'] == 99.91),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==6161) & (df_all['supply_area'] == 99.91),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==3685) & (df_all['supply_area'] == 115.70),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==3685) & (df_all['supply_area'] == 115.70),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==21288) & (df_all['supply_area'] == 116.03),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==21288) & (df_all['supply_area'] == 116.03),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==10636) & (df_all['supply_area'] == 112.40),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==10636) & (df_all['supply_area'] == 112.40),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==1162) & (df_all['supply_area'] == 154.71),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==1162) & (df_all['supply_area'] == 154.71),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==10989) & (df_all['supply_area'] == 110.51),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==10989) & (df_all['supply_area'] == 110.51),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==11096) & (df_all['supply_area'] == 97.09),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==11096) & (df_all['supply_area'] == 97.09),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==184) & (df_all['supply_area'] == 69.42),'room_count'] = 2
df_all.loc[(df_all['apartment_id']==184) & (df_all['supply_area'] == 69.42),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==534) & (df_all['supply_area'] //10 == 11.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==534) & (df_all['supply_area'] //10 == 11.0),'bathroom_count'] = 1

df_all.loc[(df_all['apartment_id']==17384) & (df_all['supply_area'] //10 == 11.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==17384) & (df_all['supply_area'] //10 == 11.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==431) & (df_all['supply_area'] //10 == 10.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==431) & (df_all['supply_area'] //10 == 10.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==4058) & (df_all['supply_area'] //10 == 9.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==4058) & (df_all['supply_area'] //10 == 9.0),'bathroom_count'] = 2

#df_all.loc[(df_all['apartment_id']==1388) & (df_all['room_count']==0)]
df_all.loc[(df_all['apartment_id']==1388) & (df_all['supply_area'] //10 == 14.0),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==1388) & (df_all['supply_area'] //10 == 14.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==7136) & (df_all['supply_area'] //10 == 7.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==7136) & (df_all['supply_area'] //10 == 7.0),'bathroom_count'] = 1

#df_all.loc[(df_all['apartment_id']==18737)]
df_all.loc[(df_all['apartment_id']==18737) & (df_all['supply_area'] //10 == 17.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==18737) & (df_all['supply_area'] //10 == 17.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==18741) & (df_all['supply_area'] >= 160),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==18741) & (df_all['supply_area'] >= 160),'bathroom_count'] = 2

#df_all.loc[(df_all['apartment_id']==18732)]

df_all.loc[(df_all['apartment_id']==18732) & (df_all['supply_area'] //10 == 11.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==18732) & (df_all['supply_area'] //10 == 11.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==18732) & (df_all['supply_area'] //10 == 18.0),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==18732) & (df_all['supply_area'] //10 == 18.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==360) & (df_all['supply_area'] //10 == 16.0),'room_count'] = 4
df_all.loc[(df_all['apartment_id']==360) & (df_all['supply_area'] //10 == 16.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==8460) & (df_all['supply_area'] //10 == 8.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==8460) & (df_all['supply_area'] //10 == 8.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==6175) & (df_all['supply_area'] > 290),'room_count'] = 5
df_all.loc[(df_all['apartment_id']==6175) & (df_all['supply_area'] > 290),'bathroom_count'] = 3

df_all.loc[(df_all['apartment_id']==6232)]
df_all.loc[(df_all['apartment_id']==6232) & (df_all['supply_area'] //10 == 19.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==6232) & (df_all['supply_area'] //10 == 19.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==15502) & (df_all['supply_area'] //10 == 10.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==15502) & (df_all['supply_area'] //10 == 10.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==568) & (df_all['supply_area'] //10 == 19.0),'room_count'] = 5
df_all.loc[(df_all['apartment_id']==568) & (df_all['supply_area'] //10 == 19.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==16837) & (df_all['supply_area'] //10 == 9.0),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==16837) & (df_all['supply_area'] //10 == 9.0),'bathroom_count'] = 2

df_all.loc[(df_all['apartment_id']==37468) & (df_all['supply_area']  <= 200),'room_count'] = 3
df_all.loc[(df_all['apartment_id']==37468) & (df_all['supply_area']  <= 200),'bathroom_count'] = 2

### 결측치 대체
* 방, 화장실은 위와 동일한 방식으로 채워넣음.
* 주차장의 경우 0으로 대체. 결측치는 0임
* 히트 및 현관의 결측치는 None으로 대체

In [17]:
### 방과 화장실 결측치
df_all.loc[df_all['apartment_id'] == 9005, ['room_count']] = 1
df_all.loc[df_all['apartment_id'] == 9005, ['bathroom_count']] = 1

df_all.loc[df_all['apartment_id'] == 1179, ['room_count']] = 4
df_all.loc[df_all['apartment_id'] == 1179, ['bathroom_count']] = 2

df_all.loc[df_all['apartment_id'] == 10627, ['room_count']] = 3
df_all.loc[df_all['apartment_id'] == 10627, ['bathroom_count']] = 1

df_all.loc[(df_all['apartment_id'] == 10627) & (df_all['supply_area'] == 56.61), ['room_count']] = 2
df_all.loc[(df_all['apartment_id'] == 10627) & (df_all['supply_area'] == 56.61), ['bathroom_count']] = 1

df_all.loc[(df_all['apartment_id'] == 7992) , ['room_count']] = 3
df_all.loc[
(df_all['apartment_id'] == 7992) & (df_all['supply_area'] <= 81), ['bathroom_count']] = 1
df_all.loc[(df_all['apartment_id'] == 7992) & (df_all['supply_area'] > 81), ['bathroom_count']] = 2

df_all.loc[(df_all['apartment_id'] == 7118) & (df_all['supply_area'] == 75.55), ['room_count']] = 3
df_all.loc[(df_all['apartment_id'] == 7118) & (df_all['supply_area'] == 75.55), ['bathroom_count']] = 1

df_all.loc[(df_all['apartment_id'] == 7118) & (df_all['supply_area']//10 == 5.0), ['room_count']] = 2
df_all.loc[(df_all['apartment_id'] == 7118) & (df_all['supply_area']//10 == 5.0), ['bathroom_count']] = 1

df_all.loc[(df_all['apartment_id'] == 4047) & (df_all['supply_area']//10 == 11.0), ['room_count']] = 3
df_all.loc[(df_all['apartment_id'] == 4047) & (df_all['supply_area']//10 == 11.0), ['bathroom_count']] =2

df_all.loc[(df_all['apartment_id'] == 37175) & (df_all['supply_area'] < 80), ['room_count']] = 2
df_all.loc[(df_all['apartment_id'] == 37175) & (df_all['supply_area'] < 80), ['bathroom_count']] = 1

df_all.loc[(df_all['apartment_id'] == 37175) & (df_all['supply_area'] > 80), ['room_count']] = 3
df_all.loc[(df_all['apartment_id'] == 37175) & (df_all['supply_area'] > 80), ['bathroom_count']] = 2

In [18]:
### 주차장 결측치
df_all.loc[(df_all['total_parking_capacity_in_site'].isnull()), ['total_parking_capacity_in_site']] = 0

In [19]:
### 히트 결측치
df_all.loc[(df_all['heat_type'].isnull()), ['heat_type']] = 'None'
df_all.loc[(df_all['heat_fuel'].isnull()), ['heat_fuel']] = 'None'

In [20]:
### 현관구조 결측치
df_all.loc[(df_all['heat_fuel'].isnull()), ['front_door_structure']] = 'None'

### 파생변수 생성

In [21]:
### 용적률(容積率)은 건축 용어로 전체 대지면적에 대한 건물 연면적의 비율을 뜻하며 백분율로 표시한다. 
### 용적률이 높을수록 건축할 수 있는 연면적이 많아져 건축밀도가 높아지므로, 적정 주거환경을 보장하기 위하여 용적률의 상한선을 지정한다.
df_all['effective_ratio'] = (df_all['exclusive_use_area'] / df_all['supply_area']) * 100

In [22]:
### 시간을 좀 더 세부적으로 나타냄. 
df_all['transaction_date1'] = df_all.transaction_date.apply(lambda x: x[-2:])
#convert int to date
df_all['transaction_year_month1'] = df_all['transaction_year_month'].astype(str)
#join month and date 
df_all['transaction_year_month_date'] = df_all[['transaction_year_month1', 'transaction_date1']].apply(lambda x: ''.join(x), axis=1)
#convert  month and date to datetime 
df_all['transaction_year_month_date'] = pd.to_datetime(df_all['transaction_year_month_date'] )
#reindext datetime
del df_all['transaction_date1']; del df_all['transaction_year_month1']

In [23]:
### 계산의 편의성을 위해 날짜를 만들어 둠.
df_all['year'] = df_all['transaction_year_month_date'].dt.year
df_all['month'] = df_all['transaction_year_month_date'].dt.month
df_all['month'] = df_all['month'].apply(lambda x : x-1 if x%2 == 0 else x)
df_all['mean_year_month'] = df_all['year']*100 + df_all['month']
del df_all['year']
del df_all['month']

In [24]:
### 동일한 아파트의 가장 최근의 거래와 현재 거래의 차이를 계산
df_all['last_month'] = df_all.groupby(['apartment_id'])['transaction_year_month'].shift(1)
df_all['diff_month'] = df_all['transaction_year_month'] - df_all['last_month']
del df_all['last_month']

### 방의 총 갯수 ( 화장실 + 방 )
df_all['total_room'] = df_all['room_count'] + df_all['bathroom_count']

### Last_price_1과 3은 동일한 아파트의 면적대비 거래액을 의미. 추가로 현재 면적을 곱해줘야 함. 
df_all['last_price_1'] = df_all['transaction_real_price']/df_all['exclusive_use_area']
df_all['last_price_1'] = df_all.groupby(['apartment_id'])['last_price_1'].shift(1)

df_all['last_price_3'] = df_all['transaction_real_price']/df_all['supply_area']
df_all['last_price_3'] = df_all.groupby(['apartment_id'])['last_price_3'].shift(1)

df_all['last_area'] = df_all.groupby(['apartment_id'])['exclusive_use_area'].shift(1)
df_all['last_transaction_year_month'] = df_all.groupby(['apartment_id'])['transaction_year_month'].shift(1)
#df_all['transaction_real_price'] = np.log1p(df_all['transaction_real_price'])

df_all['last_price_1'] = df_all['last_price_1'] * df_all['exclusive_use_area'] #현재 면적을 곱해줘서 비교를 가능하게 만듬.
df_all['last_price_3'] = df_all['last_price_3'] * df_all['supply_area'] #현재 면적을 곱해줘서 비교를 가능하게 만듬.
del df_all['last_area'],df_all['last_transaction_year_month']

### log를 씌어줘서 정규성을 띄게 만듬. 
df_all['last_price_1'] = np.log1p(df_all['last_price_1'])
df_all['last_price_3'] = np.log1p(df_all['last_price_3'])
df_all['transaction_real_price'] = np.log1p(df_all['transaction_real_price'])

### 빌딩의 간격계산
df_all['difference_building_height'] = df_all['tallest_building_in_sites'] - df_all['lowest_building_in_sites']
### 세대당 주차수 계산
df_all['capacity_per_household'] = df_all['total_parking_capacity_in_site']/df_all['total_household_count_in_sites']

### 아파트당 세대 수 계산
df_all['household_per_building'] = df_all['total_household_count_in_sites']/df_all['apartment_building_count_in_sites']

### 아파트당 타입의 비율 계산
df_all['areahousehold_per_household'] = df_all['total_household_count_of_area_type']/df_all['total_household_count_in_sites']

df_all['year'] = df_all['transaction_year_month']//100

### 거래된 기간과 완성된 년도의 차이 계산
df_all['transaction_diff_completion'] = df_all['transaction_year_month'] - df_all['year_of_completion']

### 몇번째 층인지 비율 계산
df_all['floor_ratio'] = df_all['floor']/df_all['tallest_building_in_sites']

### 재개발 예정인지 가중치 줌. 
### 35를 상한선으로 잡은것은 이 이상이 되면 재개발 될 거라는 심리가 떨어져서 임.
df_all['weight'] = 0
df_all.loc[((df_all['year']-df_all['year_of_completion']) >= 25) & ((df_all['year']-df_all['year_of_completion']) < 35) & (df_all['effective_ratio'] >= 80) & (df_all['tallest_building_in_sites'] <=5),'weight'] = 1

In [25]:
df_train.columns
df_test.columns

Index(['key', 'apartment_id', 'city', 'transaction_year_month',
       'transaction_date', 'year_of_completion', 'exclusive_use_area', 'floor',
       'latitude', 'longitude', 'address_by_law',
       'total_parking_capacity_in_site', 'total_household_count_in_sites',
       'apartment_building_count_in_sites', 'tallest_building_in_sites',
       'lowest_building_in_sites', 'heat_type', 'heat_fuel', 'room_id',
       'supply_area', 'total_household_count_of_area_type', 'room_count',
       'bathroom_count', 'front_door_structure', 'transaction_real_price'],
      dtype='object')

In [26]:
apartment = df_train.apartment_building_count_in_sites


In [27]:
df_all = df_all.reset_index(drop=True)

### 공공데이터

In [28]:
### 구청
public = df_train.total_parking_capacity_in_site


In [29]:
df_all.transaction_real_price

0          19.867600
1          19.673444
2          19.926978
3          18.805943
4          19.801963
5          19.670582
6          18.890684
7          18.197537
8          20.069339
9          19.804472
10         19.519293
11         20.040069
12         19.008467
13         20.286310
14         19.275096
15         20.901412
16         19.424982
17         18.258162
18         19.902285
19         19.320842
20         19.008467
21         19.086970
22         19.734404
23         19.819398
24         18.560443
25         20.324280
26         20.758005
27         20.584004
28         19.644456
29         18.420681
             ...    
1646306    19.552083
1646307    19.583832
1646308    19.583832
1646309    19.062535
1646310    18.997294
1646311    20.049921
1646312    19.753047
1646313    19.406498
1646314    20.161147
1646315    19.614603
1646316    19.862883
1646317    20.352202
1646318    20.094032
1646319    19.935808
1646320    20.337603
1646321    20.322788
1646322    20

### PCA

In [30]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA, FastICA,NMF,LatentDirichletAllocation,IncrementalPCA,MiniBatchSparsePCA
from sklearn.decomposition import TruncatedSVD,FactorAnalysis,KernelPCA

train_df = df_all.loc[df_all['transaction_real_price'] > 19]
test_df = df_all.loc[df_all['transaction_real_price'] <= 19 ]

train_len = train_df.shape[0]

### 날짜와 object, 공공데이터는 제거. 
train_columns = [c for c in train_df.columns if c not in ['key','transaction_real_price','transaction_year_month_date','transaction_date','heat_type','heat_fuel',
                                                          'front_door_structure']]

train_columns


['apartment_id',
 'city',
 'transaction_year_month',
 'year_of_completion',
 'exclusive_use_area',
 'floor',
 'latitude',
 'longitude',
 'address_by_law',
 'total_parking_capacity_in_site',
 'total_household_count_in_sites',
 'apartment_building_count_in_sites',
 'tallest_building_in_sites',
 'lowest_building_in_sites',
 'room_id',
 'supply_area',
 'total_household_count_of_area_type',
 'room_count',
 'bathroom_count',
 'index',
 'effective_ratio',
 'mean_year_month',
 'diff_month',
 'total_room',
 'last_price_1',
 'last_price_3',
 'difference_building_height',
 'capacity_per_household',
 'household_per_building',
 'areahousehold_per_household',
 'year',
 'transaction_diff_completion',
 'floor_ratio',
 'weight']

In [31]:
# PCA
n_comp = 1

In [32]:
# ICA
ica = FastICA(n_components=n_comp, random_state=2019)
ica2_results_train = ica.fit_transform(train_df[train_columns].fillna(-1))
ica2_results_test = ica.transform(test_df[train_columns].fillna(-1))

In [33]:
for i in range(1, n_comp+1):
    train_df['ica_' + str(i)] = ica2_results_train[:,i-1]
    test_df['ica_' + str(i)] = ica2_results_test[:, i-1]
    train_columns.append('ica_' + str(i))

df_all = pd.concat([train_df,test_df])
df_all = df_all.sort_values('index').reset_index(drop=True)

In [34]:
### 날짜 형식변경
df_all['transaction_date1'] = df_all.transaction_date.apply(lambda x: x[-2:])
#convert int to date
df_all['transaction_year_month1'] = df_all['transaction_year_month'].astype(str)
#join month and date 
df_all['transaction_year_month_date'] = df_all[['transaction_year_month1', 'transaction_date1']].apply(lambda x: ''.join(x), axis=1)
df_all['transaction_year_month_date'] = df_all['transaction_year_month_date'].astype(int)

del df_all['transaction_date1']; del df_all['transaction_year_month1']
del df_all['transaction_year_month']; del df_all['transaction_date'] ; del df_all['year']

In [35]:
train.columns

Index(['key', 'apartment_id', 'city', 'transaction_year_month',
       'transaction_date', 'year_of_completion', 'exclusive_use_area', 'floor',
       'latitude', 'longitude', 'address_by_law',
       'total_parking_capacity_in_site', 'total_household_count_in_sites',
       'apartment_building_count_in_sites', 'tallest_building_in_sites',
       'lowest_building_in_sites', 'heat_type', 'heat_fuel', 'room_id',
       'supply_area', 'total_household_count_of_area_type', 'room_count',
       'bathroom_count', 'front_door_structure', 'transaction_real_price'],
      dtype='object')

### LIGTHGBM 모델

In [36]:
### One-hot-encoding
df_all = pd.get_dummies(df_all)
train = df_all[:train_len]
df_test = df_all[train_len:]
del df_all
train = train.sort_values('index')
df_test = df_test.sort_values('index')


In [37]:
excluded_features = [
    'transaction_real_price'
]

categorical_features = [
    'apartment_id'
]

In [38]:
del train['index']
del df_test['index']
del train['mean_year_month']
del df_test['mean_year_month']

In [39]:
df_train_busan = train[train['city']==0].reset_index(drop=True)
df_test_busan = df_test[df_test['city']==0].reset_index(drop=True)
df_train_seoul = train[train['city']==1].reset_index(drop=True)
df_test_seoul = df_test[df_test['city']==1].reset_index(drop=True)

In [40]:
import time 
y_train = df_train_seoul['transaction_real_price'].reset_index(drop=True)

x_train = df_train_seoul.copy().reset_index(drop=True)
del x_train['city']; del x_train['transaction_real_price']; del x_train['last_price_3'];  del x_train['ica_1']


excluded_features = ['key','floor']
train_features = [_f for _f in x_train.columns if _f not in excluded_features]

seoul_key = df_test_seoul['key'].values
seoul_floor = df_test_seoul['floor'].values
x_test = df_test_seoul[train_features].reset_index(drop=True)

In [41]:
# LightGBM
import time
folds = KFold(n_splits=5,random_state=6,shuffle=True)
"""
n_splits : fold의 개수. 만약 n_splits=10이면 10fold가 있고, 1개가 training set 나머지 9개가 test set으로 쓰이는 것이다.

shuffle: 나누기 전에 shuffle할 건지 말 건지 선택.
데이터 순서가 임의적이지 않을 경우 shuffle =True로 하기도 한다. 
하지만 샘플들이 iid(independently and identicallly distributed) 하지 않다면 shuffling하지 않는 것이 오히려 낫다.(예.순서형 자료)

random_state: int를 부여할 경우 seed와 같은 역할을 해서 shuffle을 동일하게 한다. 
'RandomState instance'일 경우 random number를 생성한다. 'None'일 경우 np.random을 사용하여 random number를 생성한다. 
shuffle=True일 경우에 사용되는 파라미터이다.
"""

oof_preds = np.zeros(x_train.shape[0])
sub_preds = np.zeros(x_test.shape[0])

#training_set 할 fold 한개를 0으로 채워줌

start = time.time()
valid_score = 0

#시간을 표현하는 time 모듈이다. 시간대는 UTC(Universal Time Coordinated, 협정 세계시)를 사용한다.

feature_importance_df = pd.DataFrame()

#데이터 객체 생성

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(x_train, y_train)):
    trn_x, trn_y = x_train[train_features].iloc[trn_idx], y_train[trn_idx]
    val_x, val_y = x_train[train_features].iloc[val_idx], y_train[val_idx]
    
    #iloc: integer positon를 통해 값을 찾을 수 있다. label로는 찾을 수 없다 
    
    train_data = lgb.Dataset(data=trn_x, label=trn_y)
    valid_data = lgb.Dataset(data=val_x, label=val_y) 
    
    #k-fold의 속도가 늦는점을 보완하기 위해 LightGBM알고리즘 사용
    
    
    # Train the model
    params = {"objective" : "regression", "metric" : "quantile", 'n_estimators': 20000, 'early_stopping_rounds':110, 
              "num_leaves" : 30, "learning_rate" : 0.15, "bagging_fraction" : 0.9, "lambda_l1" : 0.1,
               "bagging_seed" : 0}
    #params는 부스터의 매개변수
    #'early_stopping_rounds' 검증 점수가 110 라운드 동안 향상되지 않을 때까지 훈련.
    
    
    #선형,척도, 평가(20000개로 설정), 
    
    
    lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data], verbose_eval=1000) 
    
    #verbose_eval 로 출력 비를 정해줌 
    #주어진 매개 변수로 훈련을 수행하십시오.
    
    #예측 모델 
    oof_preds[val_idx] = lgb_model.predict(val_x, num_iteration=lgb_model.best_iteration) #num_iteration ( int 또는 None , optional ( default = None ) ) - 예측에서 반복 횟수를 제한
    sub_pred = lgb_model.predict(x_test, num_iteration=lgb_model.best_iteration) / folds.n_splits
    #test할 예측 모델을 k개로 나눔 
    sub_preds += sub_pred
    
    
    #print('Fold %2d rmse : %.6f' % (n_fold + 1, np.sqrt(mean_squared_error(val_y, oof_preds[val_idx]))))
    valid_score += mean_squared_error(val_y, oof_preds[val_idx])
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_features
    fold_importance_df["importance"] = lgb_model.feature_importance() #피쳐를 가져옴
    fold_importance_df["fold"] = n_fold + 1 
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    #concat으로 feature_importance_df, fold_importance_df 두개의 데이터프레임을 합쳐줌
    
    gc.collect()
    
    """
    순환 참조 탐지 알고리즘을 수행하고 특정 세대에서 도달할 수 있는 객체(reachable)와 도달할 수 없는 객체(unreachable)를 
    구분하고 도달할 수 없는 객체 집합을 찾는다. 
    도달할 수 있는 객체 집합은 다음 상위 세대로 합쳐지고(0세대에서 수행되었으면 1세대로 이동), 
    도달할 수 없는 객체 집합은 콜백을 수행한 후 메모리에서 해제된다.
    """
print('Full rmse score %.6f\n' % np.sqrt(mean_squared_error(np.expm1(y_train), np.expm1(oof_preds))))

Training until validation scores don't improve for 110 rounds.
[1000]	training's quantile: 0.0240931	valid_1's quantile: 0.025083
[2000]	training's quantile: 0.0209753	valid_1's quantile: 0.0226361
[3000]	training's quantile: 0.0193198	valid_1's quantile: 0.0216185
[4000]	training's quantile: 0.0182026	valid_1's quantile: 0.0211075
[5000]	training's quantile: 0.0173231	valid_1's quantile: 0.0207786
[6000]	training's quantile: 0.0165894	valid_1's quantile: 0.0205472
[7000]	training's quantile: 0.0159462	valid_1's quantile: 0.0203661
[8000]	training's quantile: 0.0153783	valid_1's quantile: 0.0202557
[9000]	training's quantile: 0.0148708	valid_1's quantile: 0.020158
[10000]	training's quantile: 0.0143958	valid_1's quantile: 0.0200816
[11000]	training's quantile: 0.0139528	valid_1's quantile: 0.0200266
[12000]	training's quantile: 0.0135452	valid_1's quantile: 0.0199798
[13000]	training's quantile: 0.0131586	valid_1's quantile: 0.0199497
Early stopping, best iteration is:
[13654]	training

In [ ]:
plt.plot(trn_x, trn_y)
plt.show()

In [ ]:
plt.figure(figsize=(9, 3))
plt.plot(trn_x, trn_y)
plt.show()

In [ ]:
sns.countplot(x='apartment_id', data=df_test)
plt.show()

In [ ]:
type(trn_y)

In [ ]:
a = np.array(trn_y)

In [ ]:
b=a.reshape(-1, 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
trn_y_Y =StandardScaler().fit_transform(b)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pc = pca.fit_transform(trn_y_Y)
plt.plot(pc)
plt.show()

In [ ]:
plt.scatter(pc[:,0], pc[:,-1])

In [ ]:
plt.hist(pc, bins=20, color='green', density=True)
plt.show()

In [ ]:
type(val_y)

In [ ]:
val_numpy=np.array(val_y)
val_reshape=val_numpy.reshape(-1, 1)

In [ ]:
pca1 = PCA(n_components=1)
pc1 = pca1.fit_transform(val_reshape)
plt.plot(pc1)
plt.show()

In [ ]:
plt.hist(pc1, bins=20, color='green', density=True)
plt.show()

In [ ]:
plt.boxplot([pc, pc1], 0, vert=True)
plt.show()

In [ ]:
feature_importance_df

In [ ]:
fold_importance_df

In [ ]:
feature_importance_df['importance']